In [1]:

import os

tavily_api_key = os.environ.get('TAVILY_API_KEY', '')
api_key = os.environ.get('OPENAI_API_KEY', '')

if not api_key:
    print('You need to set the OPENAI_API_KEY environment variable to use this script.')
if not tavily_api_key:
    print('You need to set the TAVILY_API_KEY environment variable to use this script.')

In [2]:
llm_model = "gpt-4-0125-preview"

# Built in retriever

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_tool = TavilySearchResults()

In [4]:
tavily_tool.invoke("how do you get a multiball in medival madness?")

[{'url': 'https://gamefaqs.gamespot.com/pinball/915940-medieval-madness/faqs/1456',
  'content': 'Then shoot the castle to start BFTK. You will get credit for completing royal madness without making a single shot. The same applies to barnyard multiball but is much tougher to light. Hopefully you have 2 troll bombs for the finale, otherwise shooting the castle is a tough chore. Victory laps: here is a chance at some big points.'},
 {'url': 'http://www.pinball.org/rules/medievalmadness.html',
  'content': 'Castle Crusher\nTo get the "Castle Crusher" light, you must destroy the 6 Castles of the King\nof Payne and All The King\'s Men: Sir Howard Hurtz (a mobster), Francois de\nGrimm (a Frenchman), the Earl of Ego (a narcissist), the Duke of Bourbon (a\ndrunk), and Sir Psycho (a... uh, psycho).\n The third shot allows you to play the brief "The Catapult" game, where you\ncan choose which of 5 items to throw at the castle -- a duck, a cat, a cow, a\nbowling ball, and a skull head (not that S

# Accessing information from reddit

In [5]:
from langchain_community.document_loaders import RedditPostsLoader

reddit_loader = RedditPostsLoader(
    client_id="dBUWlJtE3mfzcruyAY1j4Q",
    client_secret="fRFhEWtFLmT6Hmhrn6dln3DpHQUdXA",
    user_agent="extractor by u/Master_Ocelot8179",
    categories=["new", "hot"],  # List of categories to load posts from
    mode="subreddit",
    search_queries=[
        "pinball",
    ],  # List of subreddits to load posts from
    number_posts=100,  # Default value is 10
)

In [6]:
docs = reddit_loader.load()
docs

[Document(page_content='', metadata={'post_subreddit': 'r/pinball', 'post_category': 'new', 'post_title': 'Kaneda broke the news Beetlejuice coming from Spooky with Franchi art', 'post_score': 0, 'post_id': '1bkkzl6', 'post_url': 'https://www.reddit.com/r/pinball/comments/1bkkzl6/kaneda_broke_the_news_beetlejuice_coming_from/', 'post_author': Redditor(name='robcado')}),
 Document(page_content="Hello r/Pinball community!\n\nAs a passionate player of both casual and competitive pinball, I've always been curious about the darker side of the sport: cheating. Whether it's minor rule-bending or outright foul play, I'm interested in hearing if anyone here has come across cheating during pinball competitions.\n\nHave you ever witnessed it? How was it dealt with? I'm asking out of pure curiosity and to spark a conversation about our experiences in the competitive pinball scene.\n\nLooking forward to hearing your stories!", metadata={'post_subreddit': 'r/pinball', 'post_category': 'new', 'post_t

# Building a custom Retriever

In [7]:
from bs4 import BeautifulSoup as Soup
import requests
def get_sub_urls(url):
    html_content = requests.get(url).text

    soup = Soup(html_content, "html.parser")
    links = soup.find_all('a')  # Find all anchor tags
    urls = [link.get('href') for link in links if link.get('href') is not None]
    # Add http://www.pinball.org/rules/ to all urls without http
    #urls = ["" if url.startswith('http') else 'http://www.pinball.org/rules/' + url for url in urls]
    # Remove urls that are not valid
    #urls = [url for url in urls if url.startswith('http://www.pinball.org/rules/')]

    return urls


In [8]:
pinball_url = "http://www.pinball.org/rules/index-old.html"

pinball_sub_urls = get_sub_urls(pinball_url)
pinball_sub_urls = ['http://www.pinball.org/rules/' + url for url in pinball_sub_urls if not url.startswith('http')]

In [9]:

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from bs4 import BeautifulSoup as Soup
import nest_asyncio

nest_asyncio.apply()


loader = WebBaseLoader(
    pinball_sub_urls,
    continue_on_failure=True
)


loader.requests_per_second = 2
docs = loader.aload()


Fetching pages:  78%|#######8  | 147/188 [00:30<00:08,  4.85it/s]Error fetching http://www.pinball.org/rules/starwarsepisodei2.html, skipping due to continue_on_failure=True
Error fetching http://www.pinball.org/rules/starwarstrilogy2.html, skipping due to continue_on_failure=True
Fetching pages: 100%|##########| 188/188 [00:40<00:00,  4.59it/s]


In [10]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch

vectore_store_index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=OpenAIEmbeddings(api_key=api_key),
).from_loaders([loader, reddit_loader])

/Users/tobiaslofgren/Desktop/code/venvs/flipper_ama/lib/python3.11/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [11]:
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings(api_key=api_key))


In [12]:
#retriever = vector.as_retriever()
retriever = vectore_store_index.vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['DocArrayInMemorySearch'], vectorstore=<langchain_community.vectorstores.docarray.in_memory.DocArrayInMemorySearch object at 0x1212e7490>)

In [13]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "pinball_search",
    "Search for information about Pinball. For any questions about Pinball, you must use this tool! Do not combine multiple sources of information to produce answers. If there are multiple sources that provide answers you should prompt back to the user and ask which one they want to use.",
)

In [14]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=llm_model, temperature=0,api_key=api_key)

In [15]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import LLMChain, SimpleSequentialChain

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a pinball agent, always giving as short and concise answers as possible to pinball-related questions. You do not know anything else but pinball. You can use the `pinball_search` tool to find information about pinball.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [16]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents import create_openai_functions_agent
tools = [tavily_tool, retriever_tool]
llm_with_tools = llm.bind_tools(tools)


from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

#agent = create_openai_functions_agent(llm, tools, prompt)
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [17]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [18]:

agent_executor.invoke({"input": "Hello"})



> Entering new AgentExecutor chain...
Hello! How can I assist you with pinball today?

> Finished chain.


{'input': 'Hello', 'output': 'Hello! How can I assist you with pinball today?'}

In [19]:
retriever.get_relevant_documents("how do you get a multiball in medival madness?")

[Document(page_content="Multiball\nMultiball can be started in two ways.  Locking two balls in the Main\nHabitrail will start 3-ball multiball when you launch the third ball.  The\nother way is to start Cave-In or Jaws of Life during your first run through the\nmodes.  Getting multiball this way will start a 2-ball multiball if there is\nnot a ball locked in the Main Habitrail, and 3-ball if there is a ball locked\nthere.  (Note, you don't actually have to play Cave-In or Jaws of Life.  See\nthree paragraphs down for more info.)\n\nThe object of Multiball is to collect jackpots.  The first jackpot can be\ncollected either at the Helipad or the Emergency Room.  After that, only the\nEmergency Room awards jackpots; the Helipad increases the jackpot value.  The\njackpot starts at 10M.  Each shot to the Helipad to increase the jackpot adds\n30M, and collecting a jackpot makes the next jackpot double the value of the\none just collected.  The jackpot value maxes at 300M.", metadata={'source

In [21]:
#debug
import langchain
langchain.debug = False
agent_executor.invoke({"input": "How do you get an extra ball in Indiana Jones"})



> Entering new AgentExecutor chain...

Invoking: `pinball_search` with `{'query': 'Indiana Jones extra ball'}`


It would appear that this feature is one of the buggiest on the game.
Sometimes it will give the animation for the 1B but not the points,
and it will often leave the Path of Adventure playfield confused.
Combination Shots
There are no (known) combination shots on Indiana Jones. I suspect this
is due to multiple ramps/loops having other functions (ie Dogfight and
Friends Jackpot)
Extra Ball
This game is one of the most liberal when it comes to Extra Balls. Even
on 'hard' settings you can fairly easily get 5, and on most games they
seem to be unlimited.  Also, all lit EB's are remembered between balls.

There are at least five ways of getting an EB from Indiana Jones:

Shoot Start Mode to start Raven Bar Video Mode. An Extra Ball
is available after about enough bad guys have been blown away.
Shoot it to collect.


Note: There are different Operator settings which allow diffe

{'input': 'How do you get an extra ball in Indiana Jones',
 'output': 'To get an extra ball in Indiana Jones pinball, you can:\n\n1. Shoot Start Mode to start Raven Bar Video Mode. An Extra Ball is available after enough bad guys have been blown away. Shoot it to collect.\n\nNote: The number of available Extra Balls can vary based on Operator settings. Some settings may limit the number of Extra Balls that can be collected using certain methods, while others may allow them to be unlimited.'}